CLASSIQ Assignment 6.12: Advanced Algorithms Design - Quantum Walk on a Line by Sirikarn Phuangthong

Goal: To design the quantum walk operator for the case of a line with 16 nodes, modified from from the example case of a circle with 4 nodes.

In [12]:
# To begin, we import all the required classiq packages
from classiq import *

In [3]:
# We define the number of qubits used, as well as the desired nodes in the line graph
size = 4 # how many qubits in the system, we want 16 nodes in the line graph, so we set qubit to 4
nodes = 2 ** size # number of nodes in the line graph, which corresponds to 2 ** size

In [4]:
# In order to implement the phase kickback, we need to prepare the qubite in the |−⟩ state, 
# then apply phase kickback using a diffuzer oracle. Please note both functions below.
@qfunc
def prepare_minus(x: QBit): # Prepares a qubit in the |−⟩ state using phase kickback
  X(x) # Applies a Pauli-X gate to qubit x
  H(x) # Applies a Hadamard gate to qubit x to create |−⟩ state

@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum): # Implements an oracle for phase kickback
  aux^=(x!=0) # Applies a phase flip to auxiliary qubit if x is not zero

In [5]:
# We apply the phase kickback using the auxiliary qubit and the functions above.
@qfunc
def zero_diffuzer(x: QNum): # Applies phase kickback using an auxiliary qubit
  aux = QNum('aux') # Initializes another auxiliary qubit
  allocate(1,aux) # Allocates one qubit for auxiliary qubit
  within_apply(compute=lambda: prepare_minus(aux), # Prepares the auxiliary qubit
              action=lambda: diffuzer_oracle) # Applies the oracle for phase kickback

In [6]:
# In order to implement the specific graph (in this case, a line graph), we need to define the W operator. 
# This operator will be used to apply the phase kickback to the quantum state. The W operator is defined below.
def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum): # Executes one iteration of quantum operation
    prob = [0] * nodes # Probability vectors for setting up quantum state. Here, we multiply so that there are 16 positions
    if i == 0: # This applies to the first node only so that it will immediately go to the second node
       prob[i + 1] = 1.0 
    if i == nodes - 1: # This applies to the last node only so that it will immediately go to the second to the last node
       prob[i - 1] = 1.0
    if i > 0 and i < 16 - 1: # These are the nodes in between the first and last nodes
        prob[i - 1] = 0.5
        prob[i + 1] = 0.5
    print(f'State={i}, prob vec ={prob}') # Prints current state and probability vector
    
    control(ctrl=vertices==i, # Control Condition: Vertices should equal i
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices))) # Applies phase kickback

# The W operator uses the W_iteration function to apply the phase kickback to the quantum state. 
# This is done by iterating over all the nodes in the line graph.
@qfunc 
def W_operator(vertices:QNum, adjacent_vertices: QNum): # This is similar to the 'repeat' function in qmod. Here, we interate over all the nodes.
    for i in range(nodes): # Iterates over all possible states of vertices
      W_iteration(i,vertices,adjacent_vertices) # Applies the W_iteration function

In [7]:
# The edge oracle is used to determine if the sum of the vertices and adjacent vertices is odd. 
# If the sum is odd, the output is set to 1. Otherwise, the output is set to 0.
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  res |= (((vertices+adjacent_vertices)%2) ==1) # Set res to 1 if the sum of vertices and adjacent_vertices is odd

In [8]:
# The bitwise_swap function is used to swap the qubits in the x and y arrays. 
# This is done by iterating over all the qubits in the arrays and swapping the corresponding qubits.
@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i])) # Swaps corresponding qubits in x and y arrays

In [9]:
# The S operator is defined below. 
# This operator is used to apply the edge oracle and bitwise swap to the quantum state.
@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res') # Initializes a result qubit
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices)) # Computes the res value based on edge_oracle

In [10]:
# The main code is to run the quantum operations. 
# We allocate qubits for vertices and adjacent_vertices, 
# apply the Hadamard transform to vertices, and then apply the W and S operators.
@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

  allocate(size,vertices) # Allocates qubits for vertices
  hadamard_transform(vertices) # Applies Hadamard transform to vertices to create superposition
  allocate(size,adjacent_vertices) # Allocates qubits for adjacent_vertices

  W_operator(vertices,adjacent_vertices) # Applies W_operator
  S_operator(vertices,adjacent_vertices) # Applies S_operator

In [11]:
# The code below is the standard code for creating, synthesizing, and displaying the quantum program.
qmod = create_model(main) # Creates a quantum model based on the main function
qprog = synthesize(qmod) # Synthesizes the quantum model into a quantum program
show(qprog) # Displays the synthesized quantum program

State=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0